In [1]:
import pandas as pd

In [2]:
#Files to Load
school_data_raw = "Resources/schools_complete.csv"
student_data_raw = "Resources/students_complete.csv"

#Read files into Pandas DataFrames
school_data = pd.read_csv(school_data_raw)
student_data = pd.read_csv(student_data_raw)

#Merge loaded files to make complete dataset
complete_data = pd.merge(school_data, student_data, how="left", on="school_name" )
complete_data.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [3]:
#Task 1: District Summary

#Sub-Task 1: Calculate Total Number of Schools
#Basic Method: Find #unique schools and find length (use groupby w count to preserve dataframe format)
schools_unique = complete_data.groupby(['school_name']).count()
schools_number = len(schools_unique)

#Sub-Task 2: Calculate Total Number of Students
#Basic Method: Find length of complete dataset since all rows are for individual students
students_number = len(complete_data)

#Sub-Task 3: Calculate Total Budget
#Basic Method: Sum up the budget column in school dataset
budget_total = school_data["budget"].sum()

#Sub-Task 4: Calculate Average Math Score
#Basic Method: Take average of math column in complete dataset
Math_Avg = complete_data["math_score"].mean()

#Sub-Task 5: Calculate Average Reading Score
#Basic Method: Take average of reading column in complete dataset
Read_Avg = complete_data["reading_score"].mean()

#Sub-Task 6: Calculate %Students passing Math (>=70)
#Basic Method: Locate students with passing math scores, find length, and take %
Math_Pass = len(complete_data.loc[complete_data["math_score"] >= 70])
Math_Pct = Math_Pass/students_number * 100

#Sub-Task 7: Calculate %Students passing Reading (>=70)
#Basic Method: Locate students with passing reading scores, find length, and take %
Read_Pass = len(complete_data.loc[complete_data["reading_score"]>=70])
Read_Pct = Read_Pass/students_number * 100

#Sub-Task 8: Calculate %Students passing both Math and Reading
#Basic Method: Locate students with passing math scores, then within that, passing reading scores. Find length, take % 
Math_Pass_O = complete_data.loc[complete_data["math_score"]>=70]
Math_and_Read_Pass = len(Math_Pass_O.loc[Math_Pass_O["reading_score"]>=70])
Math_and_Read_Pct = Math_and_Read_Pass/students_number * 100

#Sub-Task 9: Create dataframe with above data
District_Breakdown = pd.DataFrame({"Total #Schools": [schools_number], "Total #Students":[students_number],"Total Budget":[budget_total],"Avg Math Score":[Math_Avg],"Avg Reading Score":[Read_Avg],"Math Pass %":[Math_Pct],"Reading Pass %":[Read_Pct],"Overall Pass %":[Math_and_Read_Pct]})
District_Breakdown

#Sub-Task 10: Cleanup Formatting


,Total #Schools,Total #Students,Total Budget,Avg Math Score,Avg Reading Score,Math Pass %,Reading Pass %,Overall Pass %
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [4]:
#Task 2: Schools Summary

#Sub-Task 1: Begin assembling hw solution table from complete dataset
#Basic Method: Create subset of complete data, delete duplicate rows, create $/student budget, & sort schools by name
schools_summary = complete_data.loc[:,["school_name","type","size","budget"]]
schools_summary = schools_summary.drop_duplicates()
schools_summary["Per Student Budget"] = schools_summary['budget']/schools_summary['size']
schools_summary = schools_summary.sort_values("school_name")

#Sub-Task 2: Obtain math/reading score averages per school and merge with schools summary
#Basic Method: Obtain groupby averages of math/reading scores overall
subdata_m = complete_data.groupby('school_name')['math_score'].mean()
subdata_r = complete_data.groupby('school_name')['reading_score'].mean()
subdata_mr = pd.merge(subdata_m, subdata_r, how="left", on="school_name")
schools_summary = pd.merge(schools_summary, subdata_mr, how="left", on="school_name")

#Sub-Task 3: Calculate %students passing math, reading, and overall
#Basic Method: Filter complete dataset for passing math scores, obtain groupby counts by school, delete extra columns
#Then merge counts together into summary from previous task 2, take %, and clean-up the table

#Math
subdata_2m = complete_data.loc[complete_data["math_score"] >= 70]
subdata_2mg = subdata_2m.groupby(['school_name']).count()
subdata_2mg = subdata_2mg.drop(['School ID','type','budget','reading_score','math_score','grade','gender','Student ID','student_name'],axis=1)

#Reading
subdata_2r = complete_data.loc[complete_data["reading_score"] >= 70]
subdata_2rg = subdata_2r.groupby(['school_name']).count()
subdata_2rg = subdata_2rg.drop(['School ID','type','budget','reading_score','math_score','grade','gender','Student ID','student_name'],axis=1)

#Overall (take passing reading scores from subset of passing math scores)
subdata_2m = complete_data.loc[complete_data["math_score"] >= 70]
subdata_2o = subdata_2m.loc[subdata_2m["reading_score"] >= 70]
subdata_2og = subdata_2o.groupby(['school_name']).count()
subdata_2og = subdata_2og.drop(['School ID','type','budget','reading_score','math_score','grade','gender','Student ID','student_name'],axis=1)

#Merge counts into schools summary. How to merge >2 dataframes at once?
subdata_mr = pd.merge(subdata_2mg, subdata_2rg, how="left", on="school_name")
subdata_mro = pd.merge(subdata_mr, subdata_2og, how="left", on="school_name")
subdata_mro = subdata_mro.rename(columns={"size_x":"math","size_y":"read","size":"overall"})
schools_summary = pd.merge(schools_summary, subdata_mro, how="left", on="school_name")

#Take Percentages
schools_summary["% Passing Math"] = schools_summary['math']/schools_summary['size']*100
schools_summary["% Passing Reading"] = schools_summary['read']/schools_summary['size']*100
schools_summary["% Passing Overall"] = schools_summary['overall']/schools_summary['size']*100

#Perform dataframe cleanup
schools_summary = schools_summary.drop(['math','read','overall'], axis=1)
schools_summary


,school_name,type,size,budget,Per Student Budget,math_score,reading_score,% Passing Math,% Passing Reading,% Passing Overall
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [5]:
#Task 3: Sort and display the top five performing schools by % overall passing

schools_summary_top = schools_summary.sort_values("% Passing Overall", ascending=False)
schools_summary_top.head()

,school_name,type,size,budget,Per Student Budget,math_score,reading_score,% Passing Math,% Passing Reading,% Passing Overall
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
12,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
13,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [6]:
#Task 4: Sort and display the bottom five performing schools by % overall passing

schools_summary_bot = schools_summary.sort_values("% Passing Overall")
schools_summary_bot.head()

,school_name,type,size,budget,Per Student Budget,math_score,reading_score,% Passing Math,% Passing Reading,% Passing Overall
10,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [7]:
#Tasks 5 & 6: Create table showing math and reading score averages by grade, for each school
#Basic Method: Create conditional subset of complete dataset, groupby school and grade to get summary

grade_summary = complete_data.loc[:,["school_name","student_name","math_score","reading_score","grade"]]
grade_summary = grade_summary.groupby(['school_name','grade']).mean()
grade_summary


math_score  reading_score
school_name           grade                           
Bailey High School    10th    76.996772      80.907183
                      11th    77.515588      80.945643
                      12th    76.492218      80.912451
                      9th     77.083676      81.303155
Cabrera High School   10th    83.154506      84.253219
                      11th    82.765560      83.788382
                      12th    83.277487      84.287958
                      9th     83.094697      83.676136
Figueroa High School  10th    76.539974      81.408912
                      11th    76.884344      80.640339
                      12th    77.151369      81.384863
                      9th     76.403037      81.198598
Ford High School      10th    77.672316      81.262712
                      11th    76.918058      80.403642
                      12th    76.179963      80.662338
                      9th     77.361345      80.632653
Griffin High School   10th    84.229064      83.706897
                      11th    83.842105      84.288089
                      12th    83.356164      84.013699
                      9th     82.044010      83.369193
Hernandez High School 10th    77.337408      80.660147
                      11th    77.136029      81.396140
                      12th    77.186567      80.857143
                      9th     77.438495      80.866860
Holden High School    10th    83.429825      83.324561
                      11th    85.000000      83.815534
                      12th    82.855422      84.698795
                      9th     83.787402      83.677165
Huang High School     10th    75.908735      81.512386
                      11th    76.446602      81.417476
                      12th    77.225641      80.305983
                      9th     77.027251      81.290284
Johnson High School   10th    76.691117      80.773431
                      11th    77.491653      80.616027
                      12th    76.863248      81.227564
                      9th     77.187857      81.260714
Pena High School      10th    83.372000      83.612000
                      11th    84.328125      84.335938
                      12th    84.121547      84.591160
                      9th     83.625455      83.807273
Rodriguez High School 10th    76.612500      80.629808
                      11th    76.395626      80.864811
                      12th    77.690748      80.376426
                      9th     76.859966      80.993127
Shelton High School   10th    82.917411      83.441964
                      11th    83.383495      84.373786
                      12th    83.778976      82.781671
                      9th     83.420755      84.122642
Thomas High School    10th    83.087886      84.254157
                      11th    83.498795      83.585542
                      12th    83.497041      83.831361
                      9th     83.590022      83.728850
Wilson High School    10th    83.724422      84.021452
                      11th    83.195326      83.764608
                      12th    83.035794      84.317673
                      9th     83.085578      83.939778
Wright High School    10th    84.010288      83.812757
                      11th    83.836782      84.156322
                      12th    83.644986      84.073171
                      9th     83.264706      83.833333

In [25]:
#Task 7: Sort Test Scores and Overall Passing % by School Spend Amount
#Basic Method: Create bins, append to main table, then summarize

#Create Bins
school_spend_bins = [0,584,629,644,675]
spend_labels = ["<$584", "$585-629", "$630-644", "$645-675"]

#Attach Bins to Main Table  
schools_summary["Spend Class"] = pd.cut(schools_summary["Per Student Budget"], school_spend_bins, labels=spend_labels)

#Average by School Spend Amount
grouped_school_spend_summary = schools_summary.groupby("Spend Class").mean()

#Cleanup Table
grouped_school_spend_summary = grouped_school_spend_summary.drop(['budget','size','Per Student Budget'], axis=1)
grouped_school_spend_summary

,math_score,reading_score,% Passing Math,% Passing Reading,% Passing Overall
Spend Class,,,,,
<$584,83.455399,83.933814,93.460096,96.610877,90.369459
$585-629,81.899826,83.155286,87.133538,92.718205,81.418596
$630-644,78.518855,81.624473,73.484209,84.391793,62.857656
$645-675,76.997210,81.027843,66.164813,81.133951,53.526855


In [33]:
#Task 8: Sort Test Scores and Overall Passing % by School Size
#Basic Method: Create bins, append to main table, then summarize

#Create Bins
school_size_bins = [0,1000,2000,5000]
size_labels = ["<1000","1000-2000","2000-5000"]

#Attach Bins to Main Table
schools_summary["School Size"] = pd.cut(schools_summary["size"],school_size_bins, labels=size_labels)

#Average by School Size
grouped_school_size_summary = schools_summary.groupby("School Size").mean()

#Cleanup Table
grouped_school_size_summary = grouped_school_size_summary.drop(['budget','size','Per Student Budget'], axis=1)
grouped_school_size_summary


,math_score,reading_score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
<1000,83.821598,83.929843,93.550225,96.099437,89.883853
1000-2000,83.374684,83.864438,93.599695,96.790680,90.621535
2000-5000,77.746417,81.344493,69.963361,82.766634,58.286003


In [44]:
#Task 9: Sort Test Scores and Overall Passing % by School Type
#Basic Method: Use Groupby Averages by School Type

#Cleanup Schools_Summary for Easier Use
school_type_summary = schools_summary.drop(['size','budget','Per Student Budget','School Size'], axis=1)

#Groupby averages using School Type
grouped_school_type_summary = school_type_summary.groupby("type").mean()
grouped_school_type_summary

,math_score,reading_score,% Passing Math,% Passing Reading,% Passing Overall
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
